In [1]:
import os
import sys
from dateutil import parser
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.database.sql import psql_connection

conn = psql_connection()
%matplotlib inline

# Churn Features

We will define `churn` as anyone that was paying, and canceled their subscriptions.
Look for `customer.subscription.deleted` events in `payment_events` to define who has churned.

## Find events of churned users (after conversion, before Churn)

In [ ]:
event_query = """
SELECT 
    u.distinct_id, 
    e.type AS event_type,
    COUNT(event_id) AS event_count
FROM users AS u
INNER JOIN (
    SELECT 
        email, 
        max(churn.time) AS churned_at,
        max(converted.time) AS converted_at

    FROM customers AS c
    
    -- Churned At
    LEFT JOIN payment_events AS churn
    ON c.identifier = churn.customer_id
    
    -- Converted At
    LEFT JOIN payment_events AS converted
    ON c.identifier = converted.customer_id

    WHERE 
        churn.type = 'customer.subscription.deleted' AND 
        converted.type = 'customer.subscription.created'
    GROUP BY c.email
) AS c
ON c.email = u.email

LEFT JOIN events AS e
ON e.distinct_id = u.distinct_id
WHERE e.time < c.churned_at and e.time > c.converted_at
GROUP BY u.distinct_id, e.type;
"""

raw_event_df = pd.read_sql_query(event_query, conn)
events_df = raw_event_df.pivot(index='distinct_id', columns='event_type', values='event_count')
events_df = events_df.fillna(0)
events_df.head()

## Find age of account (in days)

In [ ]:
account_age_query = """
SELECT u.distinct_id, churn.churned_at, churn.converted_at, churn.account_age
FROM users AS u
INNER JOIN (
    SELECT 
        email, 
        max(churn.time) AS churned_at,
        max(converted.time) AS converted_at,
        extract(DAY FROM max(churn.time)-max(converted.time)) AS account_age

    FROM customers AS c
    
    -- Churned At
    LEFT JOIN payment_events AS churn
    ON c.identifier = churn.customer_id
    
    -- Converted At
    LEFT JOIN payment_events AS converted
    ON c.identifier = converted.customer_id

    WHERE 
        churn.type = 'customer.subscription.deleted' AND 
        converted.type = 'customer.subscription.created'
    GROUP BY c.email
) AS churn
ON churn.email = u.email
"""
account_age_df = pd.read_sql_query(account_age_query, conn, index_col='distinct_id')
account_age_df.head()

In [ ]:
churned_df = events_df.join(account_age_df)
churned_df.head()

In [ ]:
fig, ax = plt.subplots()
# ax.scatter(churned_df['account_age'], churned_df['Export'], alpha=0.1)
ax.hist(churned_df['Editor Opened'], bins=100)
ax.set_xlim(0, 20)
# ax.set_ylim(0, 10)

# Conversion Featrues

### Define Conversion events as events that happen before subscription.

In [6]:
converted_events_query = """
SELECT u.distinct_id, e.type, count(e.event_id)
    FROM customers AS c

    LEFT JOIN users AS u
    ON u.email = c.email

    LEFT JOIN events AS e
    ON e.distinct_id = u.distinct_id
    
    LEFT JOIN (
        SELECT customer_id, max(pe.time) AS converted_at
        FROM payment_events AS pe
        WHERE type = 'customer.subscription.created'
        GROUP BY customer_id
    ) AS conversion

    ON conversion.customer_id = c.identifier
    
    WHERE e.type IS NOT NULL AND e.time < conversion.converted_at
    GROUP BY u.distinct_id, e.type;
"""

converted_age_query = """
    SELECT 
        u.distinct_id,
        CASE WHEN COUNT(converted.customer_id) > 0 AND COUNT(churned.customer_id) = 0 THEN TRUE ELSE FALSE END AS converted,
        converted.created_at AS converted_at,
        extract(DAY FROM converted.created_at) AS account_age
    FROM customers AS c

    LEFT JOIN (
        SELECT customer_id, count(identifier), max(time) AS created_at
        FROM payment_events
        WHERE type = 'customer.subscription.created'
        GROUP BY customer_id
    ) AS converted
    ON converted.customer_id = c.identifier

    LEFT JOIN (
        SELECT customer_id, count(identifier), max(time) AS created_at
        FROM payment_events
        WHERE type = 'customer.subscription.deleted'
        GROUP BY customer_id
    ) AS churned
    ON churned.customer_id = c.identifier

    INNER JOIN users AS u
    ON u.email = c.email

    GROUP BY u.distinct_id, converted.created_at;
"""

raw_converted_events_df = pd.read_sql_query(converted_events_query, conn)
raw_converted_age_df = pd.read_sql_query(converted_age_query, conn, index_col='distinct_id')

converted_events_df = raw_converted_events_df.pivot(index='distinct_id', columns='type', values='count')
converted_df = converted_events_df.join(raw_converted_age_df).fillna(0)

In [12]:
converted_df[converted_df['converted'] == True].describe()

,App Became Active,Click Button,Click Link,Client error,Countdown Pro Button,Deck Created,Display Limit Modal,Display Limit Notification,Display Video Editor Modal,Display Welcome Countdown,...,Successfully completed pro signup,Successfully completed pro upgrade,Validation failed,View player page,Zuru Upgrade Edu Button,cancel,signin,signup,upgrade,account_age
count,1938.000000,1938.000000,1938.000000,1938.000000,1938.000000,1938.000000,1938.000000,1938.000000,1938.000000,1938.000000,...,1938.000000,1938.000000,1938.000000,1938.000000,1938.000000,1938.000000,1938.000000,1938.000000,1938.000000,1938.000000
mean,0.364809,0.911249,5.613003,3.607843,0.008256,1.141383,0.223942,2.300826,0.012384,0.077399,...,0.210010,0.163571,0.248194,0.757482,0.000516,0.021156,1.061920,0.211558,0.549020,15.878741
std,3.883771,1.126655,6.180546,28.146493,0.090510,1.897103,1.361164,7.313527,0.205378,0.395111,...,0.454159,0.369981,0.786311,2.547030,0.022716,0.157636,2.665732,0.422191,0.514048,8.595856
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000
50%,0.000000,1.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,16.000000
75%,0.000000,1.000000,7.750000,1.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,23.000000
max,104.000000,13.000000,84.000000,820.000000,1.000000,31.000000,43.000000,222.000000,7.000000,9.000000,...,9.000000,1.000000,10.000000,46.000000,1.000000,2.000000,42.000000,2.000000,3.000000,31.000000
